In [2]:
import numpy as np 
import pygame 
import math
import sys 
from copy import deepcopy 
import time 
import random 
import os
import signal

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [12]:
#Para alterar o jogo basta alterar a linha abaixo desta. Só está implementado o Ataxx, se for escolhido o GO dá erro
Game="A6x6" # "A4x4" "A5x5" "A6x6" "G7x7" "G9x9" 
SIZE = int(Game[-1])

SQUARE_SIZE = 75 #Tamanho dos quadrados da interface

PLAYER1 = 1
PLAYER2 = -1

#Cores
BLUE = (0,0,255) 
RED = (255,0,0) 
WHITE_COLOR = (255,255,255) 
BLACK_COLOR = (0,0,0) 

#Tamanhos da interface
width = SIZE * SQUARE_SIZE 
height = SIZE * SQUARE_SIZE 
width = SIZE * SQUARE_SIZE 
height = SIZE * SQUARE_SIZE 
screen_size = (width, height) 

In [13]:


class Ataxx():
    """
    Classe do jogo ataxx
    
    Argumentos:
                size -> tamanho do tabuleiro
    
    """
    def __init__(self, size):
        """
        size -> tamanho do tabuleiro
        board_matrix -> matrix que representa o estado do tabuleiro
        to_move -> jogador 
        
        
        """
        self.size = size
        self.board_matrix = np.zeros((self.size, self.size)).astype(int)
        self.to_move = PLAYER1
        #self.next = PLAYER2

        self.board_matrix[0][0] = PLAYER1
        self.board_matrix[self.size-1][self.size-1] = PLAYER1
        self.board_matrix[0][self.size-1] = PLAYER2
        self.board_matrix[self.size-1][0] = PLAYER2  


    
    def place(self, x, y): 
        self.board_matrix[x][y] = self.to_move 


    
    def jump(self, x, y, new_x, new_y): 
        self.board_matrix[x][y] = 0
        self.board_matrix[new_x][new_y] = self.to_move

        
    def calc_dist(self, x1, y1, x2, y2):
        return math.sqrt((x1-x2)**2+(y1-y2)**2)



    def play_move(self, x, y, new_x, new_y):

        if new_x < 0 or new_x >= self.size or new_y < 0 or new_y >= self.size:
            return False

        if self.board_matrix[new_x][new_y] != 0:
            return False

        dist = self.calc_dist(x, y, new_x, new_y)

        if dist == 0:
            return False
        
        elif dist < 2:
            self.place(new_x, new_y)
            return True

        elif dist < 3:
            self.jump(x, y, new_x, new_y)
            return True

        else:
            return False
    

    def is_valid_piece(self, x, y):
        if self.board_matrix[x][y] == self.to_move:
            return True
        else:
            return False

    
    def eat(self, x, y):
        for i in range(-1, 2):
            for j in range(-1, 2):
                if x+i < self.size and x+i >=0 and y+j < self.size and y+j >=0: 
                    if self.board_matrix[x+i][y+j] == -self.to_move:
                        self.board_matrix[x+i][y+j] = self.to_move
    
    
    def get_player_pieces(self, player): 
        pieces = []
        for x in range(self.size):
            for y in range (self.size): 
                if self.board_matrix[x][y] == player:
                    pieces.append([x,y])
        return pieces  
    
    
        
    def available_moves(self, player):
        moves = []
        for piece in self.get_player_pieces(player):
            x = piece[0]
            y = piece[1]
            for i in range(-3, 3):
                for j in range(-3, 3):
                    temp1_board = Ataxx(SIZE)
                    temp1_board.to_move = self.to_move
                    temp1_board.board_matrix = self.board_matrix.copy()
                    if temp1_board.play_move(x, y, x+i, y+j):
                        moves.append((temp1_board.board_matrix, (x, y), (x+i, y+j)))
        return moves 

    
    def is_game_over(self):
        if len(self.available_moves(self.to_move)) == 0 or len(self.available_moves(-self.to_move)) == 0:
            return True
        return False
    
    
    def piece_counter(self):
        player_1_counter = len(self.get_player_pieces(PLAYER1))
        player_2_counter = len(self.get_player_pieces(PLAYER2))
        return player_1_counter, player_2_counter
    
    def turn(self):
        """Keep track of the turn by flipping between BLACK and WHITE."""
        self.to_move = -self.to_move
        """
        if self.to_move == PLAYER2:
            self.to_move = PLAYER1
            #self.next = PLAYER2
        else:
            self.to_move = PLAYER2
            #self.next = PLAYER1
        """
    
                
    def draw_pieces(self, screen): 
        for k in range(SIZE):
            for l in range(SIZE):
                if self.board_matrix[k][l] == PLAYER1: 
                    pygame.draw.circle(screen, RED, (int((l+1)*SQUARE_SIZE - SQUARE_SIZE/2),int((k+1)*SQUARE_SIZE - SQUARE_SIZE/2)), SQUARE_SIZE/3)
                
                elif self.board_matrix[k][l] == PLAYER2: 
                    pygame.draw.circle(screen, BLUE, (int((l+1)*SQUARE_SIZE - SQUARE_SIZE/2),int((k+1)*SQUARE_SIZE - SQUARE_SIZE/2)), SQUARE_SIZE/3)
                
                else: 
                    pygame.draw.circle(screen, BLACK_COLOR, (int((l+1)*SQUARE_SIZE - SQUARE_SIZE/2),int((k+1)*SQUARE_SIZE - SQUARE_SIZE/2)), SQUARE_SIZE/3)
        
        pygame.display.update() 
    
    
    def draw_lines(self, screen): 
        for i in range(SIZE -  1):
            pygame.draw.line(screen, WHITE_COLOR, ((i+1)*SQUARE_SIZE,0), ((i+1)*SQUARE_SIZE,SQUARE_SIZE*SIZE), 2) 
            pygame.draw.line(screen, WHITE_COLOR, (0,(i+1)*SQUARE_SIZE), (SQUARE_SIZE*SIZE,(i+1)*SQUARE_SIZE),2)    
        pygame.display.update() 
    
    def draw_grey_circles(self, screen, x, y):
        for i in range(-2, 3):
            for j in range(-2, 3):
                if x+i >= 0 and x+i < SIZE and y+j >= 0 and y+j < SIZE:

                    if self.board_matrix[x+i][y+j] == 0:
                        pygame.draw.circle(screen, GREY_COLOR, (int((y+j+1)*SQUARE_SIZE - SQUARE_SIZE/2),int((x+i+1)*SQUARE_SIZE - SQUARE_SIZE/2)), SQUARE_SIZE/3)
                        

    def print_board(self):
       
        #os.system('cls' if os.name == 'nt' else 'clear')
        #print(self.size)
        #print(self.board_matrix.shape)
        
        for i in range(self.size): #Imprimir a matriz
            line = ""
            for j in range(self.size):
                if self.board_matrix[i][j] != -1:
                    line += " " + str(self.board_matrix[i][j]) + " "
                else:
                    line += str(self.board_matrix[i][j]) + " "

            print(line)
    
    
    
    



In [14]:
import socket
import time

def receive_data_with_timeout(agent, timeout):
    agent.settimeout(timeout)
    start_time = time.time()
    received_data = b""
    while True:
        try:
            data = agent.recv(1024)
            if data:
                received_data += data
                break
            print(received_data)
            if time.time() - start_time > timeout:
                break
        except socket.timeout:
            # Timeout occurred
            #print(f"Limite de tempo excedido pelo agente{agent}")
            return False
    return received_data.decode()






def start_server(host='localhost', port=12345):
    pygame.init()
    
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(2)

    print("Waiting for two agents to connect...")
    agent1, addr1 = server_socket.accept()
    print("Agent 1 connected from", addr1)
    bs=b'AG1 '+Game.encode()
    agent1.sendall(bs)

    agent2, addr2 = server_socket.accept()
    print("Agent 2 connected from", addr2)
    bs=b'AG2 '+Game.encode()
    agent2.sendall(bs)    

    agents = [agent1, agent2]
    current_agent = 0
    
    invalid_counter = 0
    
    jog=0
    
    #os.environ['SDL_VIDEO_WINDOW_POS'] = "%d,%d" % (100, 100)
    
    screen = pygame.display.set_mode(screen_size)   
    pygame.display.set_caption('Server')
    game.draw_lines(screen)
    game.draw_pieces(screen)
    time.sleep(10)
    #game.print_board()



    while True:
        pygame.event.pump()
        try:
            pygame.display.update()
            #print("ok")
            #game.print_board()
            #print("ok2")

            if invalid_counter >= 3:
                #print(f"agent - {1-current_agent}")
                agents[1-current_agent].sendall(b'YOU')
                current_agent = 1-current_agent
                game.turn()
                invalid_counter = 0

                continue
            if game.is_game_over(): 
                player1_counter, player2_counter = game.piece_counter()
                result = player1_counter-player2_counter

                if result > 0:
                    winner = 1
                elif result < 0:
                    winner = 2
                else:
                    winner = 0

                message = f"END {winner} {player1_counter} {player2_counter}"
                print(message)
                agents[current_agent].sendall(message.encode())
                agents[1-current_agent].sendall(message.encode())

                break

            data = receive_data_with_timeout(agents[current_agent], 3)
            if not data:
                print(f"Agent {current_agent} time limit")
                agents[1-current_agent].sendall(b'YOU')
                current_agent = 1-current_agent
                game.turn()
                invalid_counter = 0
                continue
            # Process the move (example: "MOVE X,Y")

            print(current_agent, " -> ",data)




            if is_valid_move(data):

                agents[current_agent].sendall(b'VALID')
                agents[1-current_agent].sendall(data.encode())
                            # Switch to the other agent
                game.draw_pieces(screen)
                pygame.display.update() 

                game.turn()
                current_agent = 1-current_agent
                invalid_counter = 0
                time.sleep(3)
            else:
                agents[current_agent].sendall(b'INVALID')
                invalid_counter += 1

                #print(f"INVALID COUNTER -> {invalid_counter}")






        except Exception as e:
            print("Error:", e)
            break

    print("\n-----------------\nGAME END\n-----------------\n")
    time.sleep(1)
    pygame.init()
    game_over = True
    while game_over:
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT:
                #sys.exit()

                agent1.close()
                agent2.close()
                server_socket.close()
                pygame.quit()
                game_over = False

def is_valid_move(move):
    
    # Implement the logic to check if the move is valid
    
    x1, y1, x2, y2 = int(move[5]), int(move[7]), int(move[9]), int(move[11]) 
    
    if not game.is_valid_piece(x1, y1):
        return False
    
    #print("current_agent")
    
    
    if game.play_move(x1, y1, x2, y2):
        game.eat(x2, y2)
        #game.print_board()
        
        #game.draw_pieces(screen)
        #pygame.display.update() 
        return True
    
    return False

if __name__ == "__main__":
    game = Ataxx(SIZE)
    start_server()
    


Waiting for two agents to connect...
Agent 1 connected from ('127.0.0.1', 50645)
Agent 2 connected from ('127.0.0.1', 50647)
Agent 0 time limit
1  ->  MOVE 5,0,3,2
Agent 0 time limit
1  ->  MOVE 0,5,2,5
Agent 0 time limit
1  ->  MOVE 2,5,4,5
Agent 0 time limit
1  ->  MOVE 3,2,1,4
Agent 0 time limit
1  ->  MOVE 4,5,3,4
Agent 0 time limit
1  ->  MOVE 3,4,4,3
Agent 0 time limit
1  ->  MOVE 4,5,2,5
Agent 0 time limit
1  ->  MOVE 1,4,0,2
Agent 0 time limit
1  ->  MOVE 4,3,4,4
Agent 0 time limit
1  ->  MOVE 0,2,1,1
END 2 0 8

-----------------
GAME END
-----------------

